<a href="https://colab.research.google.com/github/nars95/super-project/blob/main/task02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import packages using in this work

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, mean_squared_error
from keras.layers import InputLayer, Dense
from keras.models import Sequential
from keras.utils import plot_model, to_categorical
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

### Read the data mamographic from uci repository

In [ ]:
columns_names = ['BI-RADS','Age','Shape','Margin','Density', 'Severity']

In [ ]:
features = ['Age','Shape','Margin','Density']

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/mammographic-masses/mammographic_masses.data"
df = pd.read_csv(url, header=None, names=columns_names)

In [ ]:
df.head()

,BI-RADS,Age,Shape,Margin,Density,Severity
0,5,67,3,5,3,1
1,4,43,1,1,?,1
2,5,58,4,5,3,1
3,4,28,1,1,3,0
4,5,74,1,5,?,1


### Seems that there is not the missing values look to the information of our dataset, correct ?
Not exacly, because there is some values with the ? symbol.

Let's take a look and replace it.

In [ ]:
def missing_values(column):
  """
    Finds and counts missing values in a column.

    Args:
        column (list or pandas.Series): The column to check for missing values.

    Returns:
        tuple: A tuple containing:
            - list: A list of missing values found in the column.
            - int: The total count of missing values.

    Example:
        >>> column = ['A', 'B', '?', 'C', '?', 'D']
        >>> missing_values(column)
        (['?', '?'], 2)
    """
  list_missing_values = []
  for value in column.values:
    if value == '?':
      list_missing_values.append(value)
  return list_missing_values, len(list_missing_values)

In [ ]:
missing_values(df['Age'])

(['?', '?', '?', '?', '?'], 5)

In [ ]:
missing_values(df['BI-RADS'])

(['?', '?'], 2)

In [ ]:
missing_values(df['Shape'])

(['?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?'],
 31)

In [ ]:
missing_values(df['Margin'])

(['?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?'],
 48)

In [ ]:
missing_values(df['Density'])

(['?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?',
  '?'],
 76)

In [ ]:
missing_values(df['Severity']) # there is not missing values for the column

([], 0)

Let's replace symbol ? of each column

In [ ]:
def average_column(column):
  """
    Calculates the average of numeric values in a column.

    Args:
        column (list or pandas.Series): The column containing numeric values.

    Returns:
        int: The rounded average value of the numeric values in the column.

    Example:
        >>> column = ['10', '20', '30', '?', '40']
        >>> average_column(column)
        25
    """
  list_values = []
  for value in column.values:
    if value != '?':
      list_values.append(int(value))
  return int(np.round(np.average(list_values)))

In [ ]:
bi_rad = df['BI-RADS'].replace('?', str(average_column(df['BI-RADS'])))

In [ ]:
age = df['Age'].replace('?', str(average_column(df['Age'])))

In [ ]:
shape =df['Shape'].replace('?', str(average_column(df['Shape'])))

In [ ]:
margin = df['Margin'].replace('?', str(average_column(df['Margin'])))

In [ ]:
density = df['Density'].replace('?', str(average_column(df['Density'])))

In [ ]:
df['BI-RADS'] = bi_rad
df['Age'] = age
df['Shape'] = shape
df['Margin'] = margin
df['Density'] = density

In [ ]:
df.info() # note the the column feature continues like the object and we need transform it.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 961 entries, 0 to 960
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   BI-RADS   961 non-null    object
 1   Age       961 non-null    object
 2   Shape     961 non-null    object
 3   Margin    961 non-null    object
 4   Density   961 non-null    object
 5   Severity  961 non-null    int64 
dtypes: int64(1), object(5)
memory usage: 45.2+ KB


### Separating the df in to data ant target

In [ ]:
data = df[features].values
target = df.iloc[:, -1].values

In [ ]:
data = data.astype(np.int64) # transform the from string to int64

In [ ]:
data.dtype

dtype('int64')

In [ ]:
print(data.shape, target.shape)

(961, 4) (961,)


### Normalize the data using StandardScaler

In [ ]:
encoder = StandardScaler()
data_transform = encoder.fit_transform(data)

### Let's take a look in our data after transformation

In [ ]:
pd.DataFrame(data_transform, columns =features)

,Age,Shape,Margin,Density
0,0.797726,0.220384,1.436762,0.224804
1,-0.864912,-1.415052,-1.183216,0.224804
2,0.174237,1.038102,1.436762,0.224804
3,-1.904060,-1.415052,-1.183216,0.224804
4,1.282661,-1.415052,1.436762,0.224804
...,...,...,...,...
956,-0.587805,-0.597334,-1.183216,0.224804
957,0.035684,1.038102,1.436762,0.224804
958,0.589896,1.038102,1.436762,0.224804
959,0.728449,1.038102,1.436762,0.224804


### Split the data into train and test

In [ ]:
X , X_test, y, y_test = train_test_split(data_transform, target, train_size=0.8, random_state=42)

In [ ]:
print(X.shape, X_test.shape)

(768, 4) (193, 4)


Build cross_validation

In [ ]:
def cross_validation(model, kfold, batch_size = 32, epochs = 100, verbose=0):
  """
    Performs cross-validation for a given model using K-fold validation.

    Args:
        model (object): The machine learning model to be cross-validated.
        kfold (object): The K-fold cross-validation iterator.
        batch_size (int, optional): The number of samples per gradient update during training. Default is 32.
        epochs (int, optional): The number of epochs to train the model. Default is 100.
        verbose (int, optional): Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch. Default is 0.

    Returns:
        tuple: A tuple containing two lists of training metrics and a list of validation metrics.
            - The first list contains the average training loss for each fold.
            - The second list contains the average training accuracy for each fold.
            - The third list contains the evaluation results of the model on the validation set for each fold.
    """
  loss_train = []
  accuracy_train = []
  val_metrics = []
  train_metrics = [loss_train, accuracy_train]
  for train_index, val_index in kfold.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=verbose)
    acc = np.average(history.history['accuracy'])
    l = np.average(history.history['loss'])
    train_metrics[0].append(l)
    train_metrics[1].append(acc)
    val_metrics.append(model.evaluate(X_val, y_val))
  return (train_metrics, val_metrics)

### Build the first architecture for our model.



In [ ]:
model1 = Sequential([
    InputLayer(input_shape=(4,)),
    Dense(1024, activation='relu'),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(4, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [ ]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              5120      
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dense_3 (Dense)             (None, 128)               32896     
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dense_6 (Dense)             (None, 16)                5

In [ ]:
kfold = KFold(n_splits=10, shuffle=True)
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
metrics1 = cross_validation(model1, kfold=kfold, batch_size = 32, epochs = 100)

3/3 [==============================] - 0s 7ms/step - loss: 0.6885 - accuracy: 0.7895


In [ ]:
# average_train_accuracy for the first model in the cross_validation
average_train_accuracy1 = np.average(metrics1[0][1])
average_train_accuracy1

0.8382756298184395

In [ ]:
#vaerage_train_loss for the firt model in the cross_validation
average_train_loss1 = np.average(metrics1[0][0])
average_train_loss1

0.41282329076528546

In [ ]:
# average_accuracy in the val set during the train first model
average_val_accuracy1 = np.average(metrics1[1][1])
average_val_accuracy1 # the val_accuracy is very small

0.6953369081020355

In [ ]:
# average_vall_loss1 in the val set during the train first model
average_val_loss1 = np.average(metrics1[1][0])
average_val_loss1 # the val_loss is high yet

0.6857618987560272

In [ ]:
rate_cross_validation1 = 100*(average_train_accuracy1 - average_val_accuracy1)
rate_cross_validation1 # this rate is high that suggest the model can be overfitting in the set train or because database is so small
                       # but let's chet it on set test

14.293872171640398

Evaluate the model1 in the test set.

In [ ]:
metrics_test1 = model1.evaluate(X_test, y_test)
metrics_test1

7/7 [==============================] - 0s 4ms/step - loss: 0.4270 - accuracy: 0.8446


[0.42696431279182434, 0.8445596098899841]

In [ ]:
rate_error_accuracy1 = 100*(average_train_accuracy1 - metrics_test1[1])
rate_error_accuracy1

-0.6283980071544626

In [ ]:
rate_error_loss1 = 100*(metrics_test1[0] - average_train_loss1)
rate_error_loss1

1.4141022026538885

In [ ]:
def plot_curves(history):
  plt.plot(history.history['accuracy'], label='accuracy')
  plt.title('Accuracy and Loss')
  plt.xlabel('epochs')
  plt.ylabel('Metrics')
  plt.legend()
  plt.plot()

  plt.plot(history.history['loss'], label = 'loss')
  plt.xlabel('epochs')
  plt.ylabel('Metrics')
  plt.legend()
  plt.plot()

Building the model2 add another with 2048 neurons

In [2]:
model2 = Sequential([
    InputLayer(input_shape=(4,)),
    Dense(2048, activation='relu'),
    Dense(1024, activation='relu'),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(4, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [3]:
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2048)              10240     
                                                                 
 dense_1 (Dense)             (None, 1024)              2098176   
                                                                 
 dense_2 (Dense)             (None, 512)               524800    
                                                                 
 dense_3 (Dense)             (None, 256)               131328    
                                                                 
 dense_4 (Dense)             (None, 128)               32896     
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dense_6 (Dense)             (None, 32)                2

In [ ]:
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # increase epochs of train from 100 to 200.
metrics2 = cross_validation(model2, kfold, epochs = 200)

3/3 [==============================] - 0s 5ms/step - loss: 0.5411 - accuracy: 0.8158


In [ ]:
average_train_accuracy2 = np.average(metrics2[0][1])
average_train_accuracy2

0.8374056687951088

In [ ]:
average_train_loss2 = np.average(metrics2[0][0])
average_train_loss2

0.38403917530179027

In [ ]:
average_val_accuracy2 = np.average(metrics2[1][1])
average_val_accuracy2

0.7234573066234589

In [ ]:
average_val_loss2 = np.average(metrics2[1][0])
average_val_loss2

0.6547832190990448

In [ ]:
rate_cross_accuracy2 = 100*(average_train_accuracy2 - average_val_accuracy2)
rate_cross_accuracy2

11.394836217164993

In [ ]:
metrics_test2 = model2.evaluate(X_test, y_test)
metrics_test2

7/7 [==============================] - 0s 5ms/step - loss: 0.6400 - accuracy: 0.7876


[0.6400361657142639, 0.787564754486084]

In [ ]:
rate_error_accuracy2 = 100*(average_train_accuracy2 - metrics_test2[1])
rate_error_accuracy2

4.984091430902482

In [ ]:
rate_error_loss2 = 100*(metrics_test2[0] - average_train_loss2)
rate_error_loss2

25.599699041247366

Building the model3 using layers of regularization called dropout

In [ ]:
model3 = Sequential([
    InputLayer(input_shape=(4,)),
    Dense(2048, activation='relu'),
    keras.layers.Dropout(0.5),
    Dense(1024, activation='relu'),
    keras.layers.Dropout(0.5),
    Dense(512, activation='relu'),
    keras.layers.Dropout(0.5),
    Dense(256, activation='relu'),
    keras.layers.Dropout(0.5),
    Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),
    Dense(32, activation='relu'),
    keras.layers.Dropout(0.5),
    Dense(16, activation='relu'),
    #keras.layers.Dropout(0.5),
    Dense(8, activation='relu'),
    #keras.layers.Dropout(0.5),
    Dense(4, activation='relu'),
    #keras.layers.Dropout(0.5)
    Dense(1, activation='sigmoid')
])

In [ ]:
model3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 2048)              10240     
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense_22 (Dense)            (None, 1024)              2098176   
                                                                 
 dropout_1 (Dropout)         (None, 1024)              0         
                                                                 
 dense_23 (Dense)            (None, 512)               524800    
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 dense_24 (Dense)            (None, 256)              

In [ ]:
model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
metrics3 = cross_validation(model3, kfold, epochs = 200)

3/3 [==============================] - 0s 9ms/step - loss: 0.4767 - accuracy: 0.7632


In [ ]:
average_train_accuracy3 = np.average(metrics3[0][1])
average_train_accuracy3

0.8282646122276782

In [ ]:
average_train_loss3 = np.average(metrics3[0][0])
average_train_loss3

0.4084084689617157

In [ ]:
average_val_accuracy3 = np.average(metrics3[1][1])
average_val_accuracy3

0.8282646122276782

In [ ]:
average_val_loss3 = np.average(metrics3[1][0])
average_val_loss3

0.6663989126682281

In [ ]:
rate_cross_accuracy3 = 100*(average_train_accuracy3 - average_val_accuracy3)
rate_cross_accuracy3

13.015149858593933

In [ ]:
metrics_test3 = model3.evaluate(X_test, y_test)

7/7 [==============================] - 0s 3ms/step - loss: 0.6622 - accuracy: 0.8083


In [ ]:
rate_error_accuracy3 = 100*(average_train_accuracy3 - metrics_test3[1])
rate_error_accuracy3 # this difference between train_accuracy and test_accuracy

1.9974428683519285

In [ ]:
rate_error_loss3 = 100*(metrics_test3[0] - average_train_loss3)
rate_error_loss3

25.38041412830353